Search 이외 임베딩 활용 방법을 알려 드립니다
- use case
    - 사용자 의도 파악
    - 자주 묻는 질문 set

=> 사용자의 인풋에 따라 다른 function이 실행될 수 있는 trigger<br>

---

In [1]:
import os
import openai
from openai import OpenAI
from sklearn.cluster import KMeans
from utils import cosine_similarity

# initialize openai
os.environ['OPENAI_API_KEY']= "your_api_key_here"
openai.api_key = os.environ["OPENAI_API_KEY"]

## 1. 사용자 의도 파악
- 심리학, 기술혁신, 음악

In [26]:
psychology = [
    "인간의 의사 결정 과정에 영향을 미치는 주요 심리적 요인은 무엇인가요?",
    "색채 심리학이 사람의 감정과 행동에 어떻게 영향을 미치나요?",
    "스트레스가 인간의 신체와 정신 건강에 미치는 영향은 무엇인가요?",
    "긍정 심리학은 어떻게 사람들이 더 행복한 삶을 살 수 있도록 돕나요?",
    "대인 관계에서 비언어적 커뮤니케이션이 갖는 중요성은 무엇인가요?"
]

technology_innovation = [
    "블록체인 기술이 금융 산업 외에 어떤 분야에서 혁신을 가져오고 있나요?",
    "인공 지능의 발전이 미래의 직업 시장에 어떤 영향을 미칠까요?",
    "사물 인터넷(IoT)이 일상 생활에서 어떻게 활용될 수 있나요?",
    "가상 현실(VR)과 증강 현실(AR) 기술의 차이점은 무엇이며, 각각 어떻게 사용되나요?",
    "지속 가능한 기술 혁신을 통해 환경 문제를 해결할 수 있는 방법은 무엇인가요?"
]

music = [
    "클래식 음악과 현대 음악 사이의 주요 차이점은 무엇인가요?",
    "음악이 사람의 감정과 기억에 미치는 영향은 무엇인가요?",
    "세계 각국의 전통 음악은 그 나라의 문화와 어떻게 연결되어 있나요?",
    "음악 교육이 아이들의 인지 발달에 어떤 긍정적인 영향을 미치나요?",
    "음악 스트리밍 서비스가 음악 산업에 가져온 변화는 무엇인가요?"
]


In [27]:
def create_embeddings(txt_list):
    client = OpenAI()

    response = client.embeddings.create(
    input=txt_list,
    model="text-embedding-3-small")
    responses = [r.embedding for r in response.data]

    return responses

In [28]:
embeddings = psychology + technology_innovation + music
emb = create_embeddings(embeddings)

#### Clustering 활용

In [29]:
n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters) # Kmeans model
clusters = kmeans.fit_predict(emb)

In [30]:
clusters

array([1, 1, 1, 1, 1, 2, 2, 2, 0, 2, 0, 1, 0, 1, 0], dtype=int32)

유저가 음악에 관련된

In [38]:
input_sentence = "ai 혁신으로 미래는 어떻게 변할까요?"
sent_emb = create_embeddings([input_sentence])

In [39]:
kmeans.predict(sent_emb)

array([2], dtype=int32)

유저가 심리에 관련된 질문을 하였을때

In [33]:
input_sentence = "우을증을 앓고 있는 환자에게 어떤 치료법이 효과적일까요?"
sent_emb = create_embeddings([input_sentence])

In [35]:
kmeans.predict(sent_emb) #[1] -> psychology

array([1], dtype=int32)

#### Similarity search를 활용

In [41]:
psychology_emb = create_embeddings(psychology)
technology_innovation_emb = create_embeddings(technology_innovation)

In [42]:
def route_selection(emb_list, query_emb, threshold=0.5):
    cos_sim = [cosine_similarity(i, query_emb) for i in emb_list]

    threshold_filtered = [i for i in cos_sim if i>threshold]

    if len(threshold_filtered)>0:
        return True
    else:
        return False

In [44]:
input_sentence = "나는 감정이 너무 슬픕니다. 어떻게 해야 하나요?"
sent_emb = create_embeddings([input_sentence])

print("{} for psychology, {} for technology_innovation".format(route_selection(psychology_emb, sent_emb[0]), route_selection(technology_innovation_emb, sent_emb[0])))

False for psychology, False for technology_innovation


In [45]:
input_sentence = "AI safey에 대한 연구가 중요한 이유는 무엇인가요?"
sent_emb = create_embeddings([input_sentence])

print("{} for psychology, {} for technology_innovation".format(route_selection(psychology_emb, sent_emb[0]), route_selection(technology_innovation_emb, sent_emb[0])))

False for psychology, False for technology_innovation


In [ ]:
input_sentence = "What is the best way to learn machine learning?"
sent_emb = create_embeddings([input_sentence])

print("{} for politics, {} for machine learning".format(route_selection(politics_emb, sent_emb[0]), route_selection(ml_emb, sent_emb[0], threshold=0.4)))

Embedding을 활용하기 때문에 최소한의 input을 활용하여 clustering이 가능해짐 <br>
##### __=> 사용자의 목적을 파악하여, 각 목적에 맞는 function 실행 가능__ (guardrails 또는 semantic router)

## 2. 자주 묻는 질문 리스트

1. 동일한 방식으로 자주 묻는 질문을 카테고리 별로 저장
2. Threshold를 정해서 유사한 질문 search
3. 유사한 질문과 연결된 정보 제공

In [ ]:
password_reset = ["What steps should I take to recover my account without access to my registered email?",
                  "Is there a way to authenticate my identity for password reset without security questions?",
                  "How can I reset my password?"]
service_request = ["Are there any special offers or discounts currently available?",
                   "How can I compare the different plans to find one that suits my needs?",
                   "Where can I see user reviews or testimonials about your services?"]

---

--END--